# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 2 2022 12:57PM,243287,10,0085921320,ISDIN Corporation,Released
1,Aug 2 2022 12:57PM,243287,10,0085921319,ISDIN Corporation,Released
2,Aug 2 2022 12:57PM,243287,10,0085921331,ISDIN Corporation,Released
3,Aug 2 2022 12:57PM,243287,10,0085921364,ISDIN Corporation,Released
4,Aug 2 2022 12:57PM,243287,10,0085921370,ISDIN Corporation,Released
5,Aug 2 2022 12:57PM,243287,10,0085921371,ISDIN Corporation,Released
6,Aug 2 2022 12:57PM,243287,10,0085921381,ISDIN Corporation,Released
7,Aug 2 2022 12:57PM,243287,10,0085921387,ISDIN Corporation,Released
8,Aug 2 2022 12:57PM,243287,10,0085921341,ISDIN Corporation,Released
9,Aug 2 2022 12:57PM,243287,10,0085921372,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
50,243282,Released,6
51,243283,Released,9
52,243284,Executing,1
53,243284,Released,1
54,243287,Released,14


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243281,NaN,NaN,6.0
243282,NaN,NaN,6.0
243283,NaN,NaN,9.0
243284,NaN,1.0,1.0
243287,NaN,NaN,14.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243170,41.0,4.0,0.0
243173,0.0,0.0,16.0
243174,0.0,14.0,23.0
243204,3.0,14.0,2.0
243208,0.0,0.0,19.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243170,41,4,0
243173,0,0,16
243174,0,14,23
243204,3,14,2
243208,0,0,19


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243170,41,4,0
1,243173,0,0,16
2,243174,0,14,23
3,243204,3,14,2
4,243208,0,0,19


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243170,41,4,
1,243173,,,16
2,243174,,14,23
3,243204,3,14,2
4,243208,,,19


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 2 2022 12:57PM,243287,10,ISDIN Corporation
14,Aug 2 2022 12:50PM,243208,20,"Exact-Rx, Inc."
33,Aug 2 2022 12:39PM,243284,10,Emerginnova
35,Aug 2 2022 12:38PM,243278,16,American International Chemical
36,Aug 2 2022 12:34PM,243283,10,ISDIN Corporation
45,Aug 2 2022 12:33PM,243279,10,Bio-PRF
50,Aug 2 2022 12:33PM,243282,10,"Methapharm, Inc."
56,Aug 2 2022 12:33PM,243281,10,"Methapharm, Inc."
62,Aug 2 2022 12:32PM,243274,10,Eywa Pharma Inc.
89,Aug 2 2022 12:31PM,243280,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 2 2022 12:57PM,243287,10,ISDIN Corporation,,,14
1,Aug 2 2022 12:50PM,243208,20,"Exact-Rx, Inc.",,,19
2,Aug 2 2022 12:39PM,243284,10,Emerginnova,,1,1
3,Aug 2 2022 12:38PM,243278,16,American International Chemical,,,1
4,Aug 2 2022 12:34PM,243283,10,ISDIN Corporation,,,9
5,Aug 2 2022 12:33PM,243279,10,Bio-PRF,,,5
6,Aug 2 2022 12:33PM,243282,10,"Methapharm, Inc.",,,6
7,Aug 2 2022 12:33PM,243281,10,"Methapharm, Inc.",,,6
8,Aug 2 2022 12:32PM,243274,10,Eywa Pharma Inc.,,,27
9,Aug 2 2022 12:31PM,243280,10,ISDIN Corporation,,,27


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 12:57PM,243287,10,ISDIN Corporation,14,,
1,Aug 2 2022 12:50PM,243208,20,"Exact-Rx, Inc.",19,,
2,Aug 2 2022 12:39PM,243284,10,Emerginnova,1,1,
3,Aug 2 2022 12:38PM,243278,16,American International Chemical,1,,
4,Aug 2 2022 12:34PM,243283,10,ISDIN Corporation,9,,
5,Aug 2 2022 12:33PM,243279,10,Bio-PRF,5,,
6,Aug 2 2022 12:33PM,243282,10,"Methapharm, Inc.",6,,
7,Aug 2 2022 12:33PM,243281,10,"Methapharm, Inc.",6,,
8,Aug 2 2022 12:32PM,243274,10,Eywa Pharma Inc.,27,,
9,Aug 2 2022 12:31PM,243280,10,ISDIN Corporation,27,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 12:57PM,243287,10,ISDIN Corporation,14,,
1,Aug 2 2022 12:50PM,243208,20,"Exact-Rx, Inc.",19,,
2,Aug 2 2022 12:39PM,243284,10,Emerginnova,1,1,
3,Aug 2 2022 12:38PM,243278,16,American International Chemical,1,,
4,Aug 2 2022 12:34PM,243283,10,ISDIN Corporation,9,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 12:57PM,243287,10,ISDIN Corporation,14.0,NaN,NaN
1,Aug 2 2022 12:50PM,243208,20,"Exact-Rx, Inc.",19.0,NaN,NaN
2,Aug 2 2022 12:39PM,243284,10,Emerginnova,1.0,1.0,NaN
3,Aug 2 2022 12:38PM,243278,16,American International Chemical,1.0,NaN,NaN
4,Aug 2 2022 12:34PM,243283,10,ISDIN Corporation,9.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 12:57PM,243287,10,ISDIN Corporation,14.0,0.0,0.0
1,Aug 2 2022 12:50PM,243208,20,"Exact-Rx, Inc.",19.0,0.0,0.0
2,Aug 2 2022 12:39PM,243284,10,Emerginnova,1.0,1.0,0.0
3,Aug 2 2022 12:38PM,243278,16,American International Chemical,1.0,0.0,0.0
4,Aug 2 2022 12:34PM,243283,10,ISDIN Corporation,9.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3648983,111.0,8.0,0.0
12,486485,0.0,2.0,2.0
15,1216113,45.0,29.0,0.0
16,243278,1.0,0.0,0.0
18,243232,1.0,0.0,0.0
19,2432430,11.0,16.0,3.0
20,1945822,71.0,10.0,55.0
21,729737,1.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3648983,111.0,8.0,0.0
1,12,486485,0.0,2.0,2.0
2,15,1216113,45.0,29.0,0.0
3,16,243278,1.0,0.0,0.0
4,18,243232,1.0,0.0,0.0
5,19,2432430,11.0,16.0,3.0
6,20,1945822,71.0,10.0,55.0
7,21,729737,1.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,111.0,8.0,0.0
1,12,0.0,2.0,2.0
2,15,45.0,29.0,0.0
3,16,1.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,11.0,16.0,3.0
6,20,71.0,10.0,55.0
7,21,1.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,111.0
1,12,Released,0.0
2,15,Released,45.0
3,16,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Completed,0.0,2.0,0.0,0.0,0.0,3.0,55.0,0.0
Executing,8.0,2.0,29.0,0.0,0.0,16.0,10.0,2.0
Released,111.0,0.0,45.0,1.0,1.0,11.0,71.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Completed,0.0,2.0,0.0,0.0,0.0,3.0,55.0,0.0
1,Executing,8.0,2.0,29.0,0.0,0.0,16.0,10.0,2.0
2,Released,111.0,0.0,45.0,1.0,1.0,11.0,71.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Completed,0.0,2.0,0.0,0.0,0.0,3.0,55.0,0.0
1,Executing,8.0,2.0,29.0,0.0,0.0,16.0,10.0,2.0
2,Released,111.0,0.0,45.0,1.0,1.0,11.0,71.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()